In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from gtsam.symbol_shorthand import X
import matplotlib.pyplot as plt

from lac.slam.gtsam_factor_graph import GtsamFactorGraph
from lac.slam.slam import SLAM
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.slam.feature_tracker import FeatureTracker
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.util import load_data, load_stereo_images
from lac.params import LAC_BASE_PATH

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/stereo_lights1.0_map1_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

# Load the images
left_imgs, right_imgs = load_stereo_images(data_path)

# Load the ground truth map
map = np.load(
    Path(LAC_BASE_PATH) / "data/heightmaps/competition/Moon_Map_01_preset_0.dat",
    allow_pickle=True,
)

In [ ]:
plot_poses(poses, no_axes=True, color="black", name="Ground Truth")

In [ ]:
tracker = FeatureTracker(cam_config)

In [ ]:
# Select 2 images from similar viewpoints
img1 = left_imgs[600]
img2 = left_imgs[4300]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img1, cmap="gray")
ax[1].imshow(img2, cmap="gray")
ax[0].axis("off")
ax[1].axis("off")
plt.show()

In [ ]:
feats1 = tracker.extract_feats(img1)
feats2 = tracker.extract_feats(img2)
matches = tracker.match_feats(feats1, feats2)

points1 = feats1["keypoints"][0][matches[:, 0]].cpu().numpy()
points2 = feats2["keypoints"][0][matches[:, 1]].cpu().numpy()

In [ ]:
from lightglue import viz2d

viz2d.plot_images([img1, img2])
viz2d.plot_matches(points1, points2, lw=0.2)

# TODO: stereo matching

For each frame, triangulate stereo points. Then, match triangulated points in frame 1 with triangulated points in frame 2.
